In [1]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard


In [2]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2


In [3]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])


2022-08-08 05:41:22.655132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-08 05:41:22.655232: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [4]:
batch_size = 32
img_height = 224
img_width = 224
data_root = r"COVID_dataset"


train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 2141 files belonging to 2 classes.
Using 1927 files for training.
Found 2141 files belonging to 2 classes.
Using 214 files for validation.


In [5]:
class_names = np.array(train_ds.class_names)
print(class_names)

['COVID19' 'NORMAL']


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

2022-08-08 05:41:46.734801: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(32, 224, 224, 3)
(32,)


2022-08-08 05:41:47.916753: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

In [10]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [11]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

2022-08-08 05:41:55.116231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


(32, 1280)


In [12]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
predictions = model(image_batch)

In [14]:
predictions.shape

TensorShape([32, 2])

In [15]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [16]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10


2022-08-08 05:42:20.525039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - ETA: 0s - loss: 0.1654 - acc: 0.9351

2022-08-08 05:43:11.663363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - 66s 929ms/step - loss: 0.1654 - acc: 0.9351 - val_loss: 0.0485 - val_acc: 0.9813
Epoch 2/10
61/61 [==============================] - 35s 557ms/step - loss: 0.0442 - acc: 0.9891 - val_loss: 0.0312 - val_acc: 0.9860
Epoch 3/10
61/61 [==============================] - 33s 514ms/step - loss: 0.0311 - acc: 0.9912 - val_loss: 0.0226 - val_acc: 0.9907
Epoch 4/10
61/61 [==============================] - 29s 463ms/step - loss: 0.0234 - acc: 0.9948 - val_loss: 0.0176 - val_acc: 0.9953
Epoch 5/10
61/61 [==============================] - 28s 456ms/step - loss: 0.0184 - acc: 0.9953 - val_loss: 0.0143 - val_acc: 1.0000
Epoch 6/10
61/61 [==============================] - 28s 460ms/step - loss: 0.0151 - acc: 0.9979 - val_loss: 0.0119 - val_acc: 1.0000
Epoch 7/10
61/61 [==============================] - 28s 462ms/step - loss: 0.0126 - acc: 0.9984 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 8/10
61/61 [==============================] - 30s 473ms/step - loss: 0.010

In [17]:
t = time.time()

export_path = r"models/basic_covid/{}".format(int(t))
model.save(export_path)
model.save('model_{}.h5'.format(int(t)))

export_path

INFO:tensorflow:Assets written to: models/basic_covid/1659959808/assets


INFO:tensorflow:Assets written to: models/basic_covid/1659959808/assets


'models/basic_covid/1659959808'

In [18]:
import gradio as gr
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [23]:
model = keras.models.load_model('models/basic_covid/1659959808')


In [24]:
class_names = ['COVID19','NORMAL']
def predict_image(img):
  img_4d=img.reshape((-1,224,224,3))
    
  prediction=model.predict(img_4d).flatten()
  return {class_names[i]: float(prediction[i]) for i in range(2)}

In [ ]:
image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


2022-08-08 05:59:55.745393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
